## Hand Sign Detection
By : Reynara Ezra Pratama

### Import Library

In [1]:
import tensorflow as tf
print(tf.__version__)

2.6.0


In [2]:
!pip install ipython-autotime
%load_ext autotime

time: 218 µs (started: 2021-10-31 13:23:44 +00:00)


In [3]:
import zipfile, os

local_zip = '/content/drive/MyDrive/Data/dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/Hand Sign Detection/Dataset')
zip_ref.close()

time: 5.36 s (started: 2021-10-31 13:23:44 +00:00)


In [4]:
print(os.listdir('/tmp/Hand Sign Detection/Dataset'))

['Sign Language for Numbers']
time: 2.02 ms (started: 2021-10-31 13:23:50 +00:00)


In [5]:
type_class = os.listdir('/tmp/Hand Sign Detection/Dataset/Sign Language for Numbers')
type_class = sorted(type_class)
print(type_class)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'unknown']
time: 2.74 ms (started: 2021-10-31 13:23:50 +00:00)


In [6]:
total_data = 0
for i in type_class:
    directory = os.listdir(f'/tmp/Hand Sign Detection/Dataset/Sign Language for Numbers/{i}')
    print(f'Number of {i} Images : {len(directory)}')
    total_data += len(directory)

print(f'\nTotal Images :{total_data}')

Number of 0 Images : 1500
Number of 1 Images : 1500
Number of 2 Images : 1500
Number of 3 Images : 1500
Number of 4 Images : 1500
Number of 5 Images : 1500
Number of 6 Images : 1500
Number of 7 Images : 1500
Number of 8 Images : 1500
Number of 9 Images : 1500
Number of unknown Images : 1500

Total Images :16500
time: 28.6 ms (started: 2021-10-31 13:23:50 +00:00)


In [7]:
!pip install split_folders
import splitfolders
splitfolders.ratio('/tmp/Hand Sign Detection/Dataset/Sign Language for Numbers', 
                   '/tmp/Hand Sign Detection/Dataset/Data', 
                   seed=42, 
                   ratio=(0.8, 0.2))

Copying files: 16500 files [00:02, 5626.50 files/s]

time: 6.3 s (started: 2021-10-31 13:23:50 +00:00)


In [8]:
base_dir = '/tmp/Hand Sign Detection/Dataset/Data'
train_dir = os.path.join(base_dir, 'train') ## Train Dataset
validation_dir = os.path.join(base_dir, 'val') ## Validation Dataset

time: 2.59 ms (started: 2021-10-31 13:23:56 +00:00)


In [9]:
os.listdir('/tmp/Hand Sign Detection/Dataset/Data/train')

['4', '7', '0', '2', '3', 'unknown', '5', '9', '6', '8', '1']

time: 21 ms (started: 2021-10-31 13:23:56 +00:00)


In [10]:
os.listdir('/tmp/Hand Sign Detection/Dataset/Data/val')

['4', '7', '0', '2', '3', 'unknown', '5', '9', '6', '8', '1']

time: 5.87 ms (started: 2021-10-31 13:23:56 +00:00)


In [11]:
total_train_data = 0
for i in type_class:
    directory = os.listdir(f'/tmp/Hand Sign Detection/Dataset/Data/train/{i}')
    print(f'Number of {i} Images : {len(directory)}')
    total_train_data += len(directory)

print(f'\nTotal Images :{total_train_data}')

Number of 0 Images : 1200
Number of 1 Images : 1200
Number of 2 Images : 1200
Number of 3 Images : 1200
Number of 4 Images : 1200
Number of 5 Images : 1200
Number of 6 Images : 1200
Number of 7 Images : 1200
Number of 8 Images : 1200
Number of 9 Images : 1200
Number of unknown Images : 1200

Total Images :13200
time: 26 ms (started: 2021-10-31 13:23:56 +00:00)


In [12]:
total_test_data = 0
for i in type_class:
    directory = os.listdir(f'/tmp/Hand Sign Detection/Dataset/Data/val/{i}')
    print(f'Number of {i} Images : {len(directory)}')
    total_test_data += len(directory)

print(f'\nTotal Images :{total_test_data}')

Number of 0 Images : 300
Number of 1 Images : 300
Number of 2 Images : 300
Number of 3 Images : 300
Number of 4 Images : 300
Number of 5 Images : 300
Number of 6 Images : 300
Number of 7 Images : 300
Number of 8 Images : 300
Number of 9 Images : 300
Number of unknown Images : 300

Total Images :3300
time: 7.92 ms (started: 2021-10-31 13:23:56 +00:00)


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.2,
    horizontal_flip = True,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest'
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255,
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (64, 64),
    color_mode = 'grayscale',
    class_mode = 'categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (64, 64),
    color_mode = 'grayscale',
    class_mode = 'categorical'
)

Found 13200 images belonging to 11 classes.
Found 3300 images belonging to 11 classes.
time: 899 ms (started: 2021-10-31 13:23:56 +00:00)


In [14]:
model = tf.keras.models.Sequential([
# first CONV => RELU => CONV => RELU => POOL layer set                                   
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding="same", input_shape=(64, 64, 1)),
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding="same"),
  tf.keras.layers.MaxPool2D(2,2),
  tf.keras.layers.Dropout(0.25),

# second CONV => RELU => CONV => RELU => POOL layer set
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding="same"),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding="same"),
  tf.keras.layers.MaxPool2D(2,2),
  tf.keras.layers.Dropout(0.25),

# first (and only) set of FC => RELU layers
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.5), 

# softmax classifier
  tf.keras.layers.Dense(11, activation='softmax')
])

time: 5.66 s (started: 2021-10-31 13:23:57 +00:00)


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 16)        2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0

In [16]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'Adam',
    metrics = ['accuracy']
)

time: 13.9 ms (started: 2021-10-31 13:24:03 +00:00)


In [17]:
history = model.fit(
    train_generator,
    epochs = 50,
    validation_data = validation_generator,
    batch_size = 12,
    verbose = 1
)

Epoch 1/50
413/413 [==============================] - 50s 48ms/step - loss: 2.3617 - accuracy: 0.1311 - val_loss: 2.2121 - val_accuracy: 0.2303
Epoch 2/50
413/413 [==============================] - 19s 46ms/step - loss: 2.2150 - accuracy: 0.2036 - val_loss: 1.8554 - val_accuracy: 0.3879
Epoch 3/50
413/413 [==============================] - 19s 47ms/step - loss: 2.0547 - accuracy: 0.2702 - val_loss: 1.5866 - val_accuracy: 0.4755
Epoch 4/50
413/413 [==============================] - 19s 47ms/step - loss: 1.9116 - accuracy: 0.3188 - val_loss: 1.4020 - val_accuracy: 0.5097
Epoch 5/50
413/413 [==============================] - 19s 47ms/step - loss: 1.7792 - accuracy: 0.3705 - val_loss: 1.2892 - val_accuracy: 0.5576
Epoch 6/50
413/413 [==============================] - 19s 47ms/step - loss: 1.6836 - accuracy: 0.4030 - val_loss: 1.2675 - val_accuracy: 0.5239
Epoch 7/50
413/413 [==============================] - 19s 47ms/step - loss: 1.6062 - accuracy: 0.4315 - val_loss: 1.0918 - val_accuracy:

In [19]:
import json
model_json = model.to_json()
with open("model_in_json.json", "w") as json_file:
    json.dump(model_json, json_file)

time: 16.2 ms (started: 2021-10-31 13:41:03 +00:00)


In [20]:
model.save_weights('model_weights.h5')

time: 39.3 ms (started: 2021-10-31 13:41:06 +00:00)
